# Export Subset of OpenQuake Results

Sometimes `--export-outputs` produces way too much data, and it's handy to zip up the result.

In [3]:
import os
import sys
import tarfile
import subprocess

In [4]:
# configuration
CALC_ID = 111
export_dir = '../HazardOutputs'
result_dir = os.path.join(export_dir, 'calc_%d' % CALC_ID)
all_outputs_file = os.path.join(result_dir, 'all_outputs.txt')
output_keywords = ['Hazard Curves', 'Hazard Curve (multiple imts)',
                   'Hazard Map', 'UHS']
type_keywords = ['mean', 'quantile']

In [5]:
# generate listing of available results
command = ['oq-engine', '--lo', str(CALC_ID)]
print('Listing outputs with "%s > %s"' 
      % (' '.join(command), all_outputs_file))
sys.stdout.flush()
with open(all_outputs_file, 'w') as lho_file:
    subprocess.call(command, stdout=lho_file, stderr=subprocess.STDOUT)

Listing outputs with "oq-engine --lo 111 > ../HazardOutputs/calc_111/all_outputs.txt"


In [6]:
# export results of interest
exported_files = []
n_exported, n_results = 0, 0
with open(all_outputs_file, 'r') as lho_file:
    for line in lho_file:
        try:
            result_id = int(line.split()[0])
            n_results += 1
        except ValueError:
            print line.strip()
            continue
        
        is_output = any([key.lower() in line.lower() 
                         for key in output_keywords])
        is_type = any([key.lower() in line.lower() 
                       for key in type_keywords])
        if is_output and is_type:
            print line.strip()
            sys.stdout.flush()
            command = ['oq-engine', '--eo', str(result_id), export_dir]
            pipes = subprocess.Popen(command, 
                                     stdout=subprocess.PIPE, 
                                     stderr=subprocess.PIPE)
            std_out, std_err = pipes.communicate()
            if pipes.returncode != 0:
                err_msg = "%s\nReturn Code: %s" % (std_err.strip(), 
                                                   pipes.returncode)
                raise Exception(err_msg)
            elif len(std_err):
                print(std_err.strip())
            
            exported_file = std_out.replace('File Exported:','').strip()
            exported_files.append(os.path.relpath(exported_file))

print('Exported %d out of %d results' % (len(exported_files), n_results))

id | output_type | name
17250 | Hazard Curve | 0.01 quantile Hazard Curves PGA
17254 | Hazard Curve | 0.01 quantile Hazard Curves SA(0.2)
17258 | Hazard Curve | 0.01 quantile Hazard Curves SA(1.0)
17251 | Hazard Curve | 0.5 quantile Hazard Curves PGA
17255 | Hazard Curve | 0.5 quantile Hazard Curves SA(0.2)
17259 | Hazard Curve | 0.5 quantile Hazard Curves SA(1.0)
17252 | Hazard Curve | 0.99 quantile Hazard Curves PGA
17256 | Hazard Curve | 0.99 quantile Hazard Curves SA(0.2)
17260 | Hazard Curve | 0.99 quantile Hazard Curves SA(1.0)
17249 | Hazard Curve | Mean Hazard Curves PGA
17253 | Hazard Curve | Mean Hazard Curves SA(0.2)
17257 | Hazard Curve | Mean Hazard Curves SA(1.0)
17245 | Hazard Curve (multiple imts) | mean-curves-multi-imt
17246 | Hazard Curve (multiple imts) | quantile(0.01)-curves
17247 | Hazard Curve (multiple imts) | quantile(0.5)-curves
17248 | Hazard Curve (multiple imts) | quantile(0.99)-curves
28926 | Hazard Map | 0.01 Quantile Hazard Map(0.00210499170414) PGA
289

In [7]:
print('Compressing: tar -hzcvf "%s.tar.gz" "%s"' % 
      (result_dir, result_dir))
with tarfile.open(result_dir + '.tar.gz', 'w:gz') as tar:
    tar.add(result_dir, arcname=os.path.split(result_dir)[1])

Compressing: tar -hzcvf "../HazardOutputs/calc_111.tar.gz" "../HazardOutputs/calc_111"
